In [574]:
import sys
import pickle
import json
import re
import pandas as pd
import ast
import numpy as np
sys.path.append('../')
from sklearn.pipeline import Pipeline
from modules.transformers.CenterPosition import CenterPositionTransform
from modules.transformers.TextType import isdateTransform, isfloatTransform,istextTransform
from modules.transformers.textalignment import AlignmentTransform
pd.set_option('display.max_rows', 1000)

In [ ]:
def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [575]:
SYMBOL = {
    "$": 0,
}
sfea_words = SYMBOL

def clean_word(word):
    """
    Clean up a word
    :param word: input wordText
    :return: cleaned up word string
    """
    cword = word.strip(' /)*%-_:;\'\"')
    cword = cword.replace(',', '')
    cword = cword.replace('.', '')
    cword = cword.replace('(', '')
    cword = cword.replace('$', '')
    return cword
def gen_raw_feature(words):
    """
    Input a word list from one page, output preliminary features for this page.
    :param words: A list of strings which are words from this page.
    :return: A list of preliminary features contains word frequencies and other features like word count.
                Need some cleanup afterwards.
    """
    word_count = 0
    number_count = 0
    fea_sym = [0]*(max(sfea_words.values())+1)
    for w in words:
        # Count '$'
        dn = w.count('$')
        fea_sym[sfea_words['$']] += dn
        if dn > 1:
            wl = w.split('$')
            w = wl[0]
            words += wl[1:]
        cw = clean_word(w)
        if not cw:
            continue
        # Increase word count
        word_count += 1
        # Check if numbers and increase number count
        if cw.isnumeric():
            number_count += 1
            continue
    # Construct other features
    if not word_count:
        fea_oth = [0, 0, 0]
    else:
        fea_oth = [word_count, number_count]
    return fea_sym + fea_oth

In [576]:
def read_pages(ticker):
    with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+ticker+'\\all.json', 'r') as f:
        datastore = json.load(f)
    df = pd.DataFrame()
    for i in range(0, len(datastore['Pages'])):
        df = df.append(pd.DataFrame(datastore['Pages'][i]))
    df = df[df.index == 0].set_index('PageNumber')
    df.index = df.index.astype(int)
    df = df.sort_index()
    df['Dollar_Count'] = ''
    df['Word_Count'] = ''
    df['Number_Count'] = ''
    for i in range(0, len(df)):
        features = gen_raw_feature(df.WordText[i])
        df['Dollar_Count'][i] = features[0]
        df['Word_Count'][i] = features[1]
        df['Number_Count'][i] = features[2]
    return df

In [577]:
def read_words(ticker):
    with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+ticker+'\\all.json', 'r') as f:
        datastore = json.load(f)
    df = pd.DataFrame()
    for i in range(0, len(datastore['Pages'])):
        df = df.append(pd.DataFrame(datastore['Pages'][i]))
    df = df[df.index != 0].set_index('PageNumber')
    df.index = df.index.astype(int)
    df = df.sort_index()
    return df

In [578]:
def predict_pages(df):
    array = [df.Number_Count/df.Word_Count, df.Dollar_Count]
    g = number_clf.predict_proba(np.transpose(array))
    m = text_clf.predict_proba(df.WordText)
    all_probas = np.hstack((m, g))
    all_probas
    return final_clf.predict(all_probas)

In [579]:
with open('text_clf.pickle', 'rb') as f:
    text_clf = pickle.load(f)
with open('number_clf.pickle', 'rb') as f:
    number_clf = pickle.load(f)
with open('final_clf.pickle', 'rb') as f:
    final_clf = pickle.load(f)

In [580]:
chk = read_pages('DIS')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [581]:
lister = predict_pages(chk)
index = []
financials = [index.append(i) for i in range(0,len(lister)) if lister[i]==1]
index

[1, 2, 3, 4, 5]

In [582]:
chk_words = read_words('DIS').reset_index()
chk_words = chk_words[chk_words.PageNumber.isin(index)]
chk_words = chk_words.reset_index()
chk_words['index'] = 1

In [583]:
intakePipeSteps = []
intakePipeSteps.append(('CenterTransform',CenterPositionTransform()))
intakePipeSteps.append(('isfloat',isfloatTransform('WordText')))
intakePipeSteps.append(('isdate',isdateTransform('WordText')))
intakePipeSteps.append(('istext',istextTransform('WordText')))
combined = Pipeline(intakePipeSteps)
combined

Pipeline(steps=[('CenterTransform', <modules.transformers.CenterPosition.CenterPositionTransform object at 0x00000198B0F1B240>), ('isfloat', <modules.transformers.TextType.isfloatTransform object at 0x00000198B0F1B358>), ('isdate', <modules.transformers.TextType.isdateTransform object at 0x00000198B0F1B438>), ('istext', <modules.transformers.TextType.istextTransform object at 0x00000198B0F1B4A8>)])

In [584]:
date = combined.named_steps['isdate'].transform(chk_words)
floater = combined.named_steps['isfloat'].transform(chk_words)
text = combined.named_steps['istext'].transform(chk_words)
center = combined.named_steps['CenterTransform'].transform(chk_words)

802 7 isdate in
802 9 isfloat in
802 8 istext in
802 9 isfloat in
802 10 centerposition in


In [733]:
chk_words = chk_words.sort_values(['PageNumber','Y_Min','X_Min'])
chk_words['WordText'] = chk_words['WordText'].str.replace(r",","")

In [734]:
def text_align(df, length):
    newlist = []
    previous = 0
    count = 0
    for row in df:
        if count == 0:
            previous = row
            newlist.append(previous)
        else:
            if abs(row - previous) > length:
                previous = row
                newlist.append(row)
            else:
                row = previous
                newlist.append(row)
        count = count + 1
    return newlist

In [802]:
line_by_y = chk_words
line_by_y['Y_Min'] = text_align(line_by_y.Y_Min, 40)

In [803]:
lines = chk_words.sort_values('X_Min').groupby(['PageNumber','Y_Min'])['WordText'].apply(lambda x: "%s" % ', '.join(x)).to_frame()

In [804]:
lines = lines.reset_index()

In [805]:
lines

,PageNumber,Y_Min,WordText
0,1,203.0,"PART, I., FINANCIAL, INFORMATION"
1,1,298.0,"Item, 1:, Financial, Statements"
2,1,373.0,"THE, WALT, DISNEY, COMPANY"
3,1,417.0,"CONDENSED, CONSOLIDATED, STATEMENTS, OF, INCOME"
4,1,464.0,"(unaudited;, in, millions, except, per, share,..."
5,1,536.0,"Quarter, Ended"
6,1,589.0,"December, 2017, 30, December31, 2016"
7,1,680.0,Revenues:
8,1,726.0,"Services, 12984, 12406"
9,1,778.0,"Products, 2367, 2378"


In [806]:
column_by_year = chk_words[chk_words.WordText.str.contains("20") & (chk_words.WordText.str.len() == 4) & chk_words.WordText_isfloat == True]
column_by_year

,index,PageNumber,WordText,X_Max,X_Min,Y_Max,Y_Min,WordText_isdate,WordText_isfloat,WordText_istext,X_Center,Y_Center
96,1,1,2017,1874.0,1807.0,653.0,589.0,1,True,0,1840.5,639.5
98,1,1,2016,2221.0,2160.0,653.0,589.0,1,True,0,2190.5,639.5
232,1,2,2017,1863.0,1798.0,517.0,453.0,1,True,0,1830.5,505.0
229,1,2,2016,2218.0,2158.0,516.0,453.0,1,True,0,2188.0,504.5
461,1,3,2017,2209.0,2151.0,391.0,327.0,1,True,0,2180.0,378.5
464,1,3,2017,1853.0,1786.0,390.0,368.0,1,True,0,1819.5,379.0
653,1,4,2017,1844.0,1777.0,482.0,417.0,1,True,0,1810.5,469.5
656,1,4,2016,2207.0,2147.0,482.0,417.0,1,True,0,2177.0,469.5


In [807]:
columndf = pd.DataFrame()
for index,row in column_by_year.iterrows():
    dic = {}
    cleanunder = []
    textover = []
    textunder = []
    for index1,row1 in chk_words.iterrows():
        if row['PageNumber'] == row1['PageNumber'] and row1['X_Center'] > (row['X_Center']-100) and row1['X_Center'] < (row['X_Center']+300):           
            if row1['Y_Min'] > row['Y_Min']:
                clean = re.sub('[^0-9.]','', row1['WordText'])
                nodecimal = re.findall(r'\d+\.\d+',clean)
                if len(nodecimal) == 0:
                    clean = re.findall(r'\d+',clean)
                else:
                    clean = nodecimal
                try:
                    clean[0] = float(clean[0])
                    cleanunder.append(clean[0])
                except:
                    pass
        if row['PageNumber'] == row1['PageNumber'] and row1['X_Center'] > (row['X_Center']-200) and row1['X_Center'] < (row['X_Center']+150):
            if row1['Y_Center']+100 > (row['Y_Center']) and row1['Y_Center'] <= (row['Y_Center']):
                textover.append(row1['WordText'])
    dic['Page'] = row['PageNumber']
    dic['Year'] = row['WordText']
    dic['Values'] = cleanunder
    dic['Title'] = textover
    columndf = columndf.append(dic, ignore_index=True)

In [808]:
linescopy = lines
linescopy.WordText = lines.WordText.replace(',','')
linescopy.WordText = lines.WordText.str.split()

In [809]:
finaldf = pd.DataFrame()
rowdic = {}
for index,row in linescopy.iterrows():
    for index1,row1 in columndf.iterrows():
         if row['PageNumber'] == row1['Page']:
            for item in row.WordText:
                notext = re.sub('[^0-9.]','', item)
                nodecimal = re.findall(r'\d+\.\d+',notext)
                if len(nodecimal) == 0:
                    notext = re.findall(r'\d+',notext)
                else:
                    notext = nodecimal
                try:
                    clean = float(notext[0])
                    if clean in row1['Values']:
                        novalues = [re.sub('[\\\,()$]', '', s) for s in row.WordText if is_float(re.sub('[\\\,()$]', '', s)) == False]
                        rowdic['Value'] = clean
                        rowdic['Description'] = ' '.join(novalues)
                        rowdic['Title'] = ' '.join(row1['Title'])
                        rowdic['Page'] = row['PageNumber']
                        finaldf = finaldf.append(rowdic, ignore_index=True)
#                         print(clean, ' '.join(novalues), ' '.join(row1['Title']))
                except:
                    pass

In [810]:
finaldf

,Description,Page,Title,Value
0,Services,1.0,Quarter December 2017 30,12984.000
1,Services,1.0,Ended December31 2016,12406.000
2,Products,1.0,Quarter December 2017 30,2367.000
3,Products,1.0,Ended December31 2016,2378.000
4,Total revenues,1.0,Quarter December 2017 30,15351.000
5,Total revenues,1.0,Ended December31 2016,14784.000
6,Cost of services exclusive ofdepreciation and ...,1.0,Quarter December 2017 30,7334.000
7,Cost of services exclusive ofdepreciation and ...,1.0,Ended December31 2016,7020.000
8,Cost of products exclusive ofdepreciation and ...,1.0,Quarter December 2017 30,1403.000
9,Cost of products exclusive ofdepreciation and ...,1.0,Ended December31 2016,1386.000
